## Monte Carlo Simulations

Parallelization tips on this notebook are borrowed from Julia's [documentation](http://docs.julialang.org/en/stable/manual/parallel-computing/).

### A Small Example

Let's try to simulate

\begin{align*}
  \begin{aligned}
    x_{k+1} = \begin{bmatrix}
      0.5 & 0 \\
      0 & 1.5
    \end{bmatrix} x_{k} + \begin{bmatrix}
      1 \\ 1
    \end{bmatrix} u_{k} + n_{k} \,,
  \end{aligned}
\end{align*}

with the controller $u_{k} = \begin{bmatrix}0.16 & -1.96\end{bmatrix} x_{k}$ and standard, normal noise $n_{k}$.

### Writing the Functions

Since Julia is a JIT-compiled language, writing a function is almost always beneficial (for efficiency purposes).

#### Get Some Workers

First, let's allocate some workers for our jobs.

In [1]:
addprocs(3) # Add 3 workers

3-element Array{Int64,1}:
 2
 3
 4

#### The Kernel

This is the main function to do the computations.

In [2]:
@everywhere function simulate_kernel!(x, range; x₀ = zeros(2))
    A = [0.5 0; 0 1.5]
    B = [1; 1]
    K = [0.16 -1.96]
    Ã = A + B*K
    @show range
    for n in range
        x[:,1,n] = x₀
        for k in 2:size(x,2)
            x[:,k,n] = Ã*x[:,k-1,n] + randn(2)
        end
    end
    x
end

#### Serial Implementation

Serial implementation just does all the computations in the kernel, serially.

In [3]:
simulate_serial!(x; x₀ = zeros(2)) = simulate_kernel!(x, indices(x,3); x₀ = x₀)

simulate_serial! (generic function with 1 method)

#### Parallel Implementation

Parallel implementation has a small change in the outermost for loop --- `@sync` and `@parallel` macro.

In [4]:
function simulate_parallel!(x; x₀ = zeros(2))
    A = [0.5 0; 0 1.5]
    B = [1; 1]
    K = [0.16 -1.96]
    Ã = A + B*K
    @sync @parallel for n in indices(x,3)
        x[:,1,n] = x₀
        for k in 2:size(x,2)
            x[:,k,n] = Ã*x[:,k-1,n] + randn(2)
        end
    end
    x
end

simulate_parallel! (generic function with 1 method)

#### Benchmark

Let's do some benchmarking.

Serial implementation first.

In [5]:
N = 10000 # Monte Carlo simulations
x = zeros(2,100,N)

simulate_serial!(x; x₀ = [50; 100]); # run once to compile the code, first

@time simulate_serial!(x; x₀ = [50; 100]);

range = Base.OneTo(10000)
range = Base.OneTo(10000)
  0.703533 seconds (9.82 M allocations: 467.279 MB, 7.00% gc time)


In [ ]:
x[:,:,1]

In [ ]:
x[:,:,2]

In [8]:
x = zeros(2,100,N)

simulate_parallel!(x; x₀ = [50; 100]);

@time simulate_parallel!(x; x₀ = [50; 100]);

  0.367858 seconds (931 allocations: 42.313 KB)


In [ ]:
x[:,:,1]

In [ ]:
x[:,:,2]

In [11]:
x = SharedArray(Float64, (2,100,N))

simulate_parallel!(x; x₀ = [50; 100])

@time simulate_parallel!(x; x₀ = [50; 100]);

  0.294646 seconds (1.53 k allocations: 55.622 KB)


In [ ]:
x[:,:,1]

In [ ]:
x[:,:,2]

#### Manual Parallelization

One can also parallelize the computations manually, having a linear, balanced distribution of the work.

In [14]:
@everywhere function mychunk(q::SharedArray)
    idx = indexpids(q)
    idx == 0 && return 1:0 # master does not have any work
    nchunks = length(procs(q))
    splits = [round(Int,s) for s in linspace(0, size(q,3), nchunks + 1)]
    return splits[idx]+1:splits[idx+1]
end

In [15]:
@everywhere simulate_job!(x; x₀ = zeros(2)) = simulate_kernel!(x, mychunk(x); x₀ = x₀)

In [16]:
function simulate_shared!(x; x₀ = zeros(2))
    @sync for p in procs(x)
        @async remotecall_wait(simulate_job!, p, x; x₀ = x₀)
    end
end

simulate_shared! (generic function with 1 method)

In [17]:
x = SharedArray(Float64, (2,100,N))

simulate_shared!(x; x₀ = [50; 100])

@time simulate_shared!(x; x₀ = [50; 100])

	From worker 3:	range = 3334:6667
	From worker 4:	range = 6668:10000
	From worker 2:	range = 1:3333
	From worker 2:	range = 1:3333
	From worker 3:	range = 3334:6667
	From worker 4:	range = 6668:10000
  0.293813 seconds (890 allocations: 79.375 KB)


In [ ]:
x[:,:,1]

In [ ]:
x[:,:,2]

### Saving the Data

Data can be saved in binary format (preferred way due to precision) or in delimited (plain) text format.

#### Binary Format

In [20]:
write("results-$(size(x,1))-$(size(x,2))-$(size(x,3)).dat", x)

16000000

In [21]:
y = read("results-2-100-10000.dat", Float64, (2,100,10000));

In [22]:
y[:,:,1] == x[:,:,1]

true

#### Plain Text Format

In [23]:
open("results-$(size(x,1))-$(size(x,2))-$(size(x,3)).csv", "w") do io
    n, m, k = size(x)
    
    x̄ = transpose(reshape(mean(x, 3), n, m))
    x̃ = transpose(reshape(var(x, 3), n, m))
    
    header = ["x_1_m" "x_2_m" "x_1_v" "x_2_v"]
    
    writecsv(io, [header; [x̄ x̃]])
end

In [24]:
y, _ = readcsv("results-2-100-10000.csv"; header = true);

In [25]:
y[:,1] == mean(x, 3)[1,:,1]

true